In [1]:
import os
import json
import pandas as pd
import numpy as np
import folium
import branca.colormap as cm

In [2]:
df = pd.read_csv(os.path.join('data','state_usage.csv'))
df = df[['state', 'December 2008 YTD', 'December 2011 YTD', 'December 2014 YTD', 'December 2017 YTD']]
names = ['December 2008 YTD', 'December 2011 YTD', 'December 2014 YTD', 'December 2017 YTD']
df = df.drop([22]).reset_index(drop=True)
us_states = os.path.join('data', 'us-states.json')
geo_json_data = json.load(open(us_states))
df['2008'] = np.log(df['December 2008 YTD'])
df['2011'] = np.log(df['December 2011 YTD'])
df['2014'] = np.log(df['December 2014 YTD'])
df['2017'] = np.log(df['December 2017 YTD'])
# df
total_min = df[['2008', '2011', '2014', '2017']].min().min()
total_max = df[['2008', '2011', '2014', '2017']].max().max()
print(total_min, total_max)

7.63867982388 13.0219572352


In [3]:
def plot_map(df, state_col, year_col, geo_json_data, output_path='out.html'):
#     vmin = df[year_col].min()
#     vmax = df[year_col].max()
    vmin = total_min
    vmax = total_max
    linear = cm.linear.Paired.to_step(5).scale(vmin,vmax)
    df_dict = df.set_index(state_col)[year_col]
    m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)
    folium.GeoJson(
        geo_json_data,
        style_function=lambda feature: {
            'fillColor': linear(df_dict[feature['id']]),
            'color': 'black',
            'weight': 2,
            'dashArray': '5, 5'
        }
    ).add_to(m)
    colormap=cm.linear.Paired.to_step(5).scale(vmin,vmax)
    colormap.caption = 'Electricity Net Generation (log GWH) at ' + year_col
    m.add_children(colormap)
    m.save(output_path)
    return m

In [4]:
m2008 = plot_map(df, 'state', '2008', geo_json_data, 'result/2008.html')
m2011 = plot_map(df, 'state', '2011', geo_json_data, 'result/2011.html')
m2014 = plot_map(df, 'state', '2014', geo_json_data, 'result/2014.html')
m2017 = plot_map(df, 'state', '2017', geo_json_data, 'result/2017.html')

/Users/wkw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [5]:
# m2008
# m2011
# m2014
m2017